In [1]:
import pandas as pd
import numpy as np
#from CRPS import CRPS
from sklearn.metrics import mean_pinball_loss

In [2]:
predictions_model1 = pd.read_csv("../../results/predictions_model1.csv")
predictions_model2 = pd.read_csv("../../results/predictions_model2.csv")
predictions_model3 = pd.read_csv("../../results/predictions_model3.csv")
predictions_GB1 = pd.read_csv("../../results/predictions_GB1.csv")
predictions_GB2 = pd.read_csv("../../results/predictions_GB2.csv")

In [3]:
# import data
hour = 17
data = pd.read_csv(f"../../data/final_{hour}.csv")
data["datetime"] = pd.to_datetime(data["datetime"])

In [4]:
# cleanup
data.drop(["loadFR"], axis=1, inplace=True)
data.dropna(inplace=True)
data.drop_duplicates(subset="datetime", keep="last", inplace=True)

In [5]:
# train/test split
data = data[data["datetime"].dt.year < 2022]
data_train = data[data["datetime"].dt.year < 2021]
data_test = data[(data["datetime"].dt.year == 2021)]
X_train = data_train.drop(["datetime", "priceBE"], axis=1)
y_train = data_train["priceBE"]
X_test = data_test.drop(["datetime", "priceBE"], axis=1)
y_test = data_test["priceBE"]

In [6]:
def CRPS(predictions, observations):
    results = pd.DataFrame([])
    for column in predictions:
        delta = np.array(predictions[column].values - observations.values)
        tau = (predictions.columns.get_loc(column)+1)/100
        results[predictions[column].name] = np.square(tau - np.heaviside(delta, 1))
    return results.sum(axis=1).mean()

In [19]:
predictions_model1

,tau= 0.01,tau= 0.02,tau= 0.03,tau= 0.04,tau= 0.05,tau= 0.06,tau= 0.07,tau= 0.08,tau= 0.09,tau= 0.10,...,tau= 0.90,tau= 0.91,tau= 0.92,tau= 0.93,tau= 0.94,tau= 0.95,tau= 0.96,tau= 0.97,tau= 0.98,tau= 0.99
0,23.075702,28.581113,30.751707,30.957551,31.597426,32.013928,33.193387,34.077713,35.794869,36.422584,...,67.062367,68.556864,69.044940,69.251869,71.109009,76.364204,77.802353,82.040597,85.992581,135.985530
1,26.146313,31.751622,35.332234,37.558750,37.960081,38.827664,39.636272,40.273952,41.651207,42.405983,...,68.267580,68.143125,69.255114,71.345904,71.895948,78.159868,78.725953,85.689150,96.431235,152.248902
2,10.613494,24.902055,29.027069,30.331820,30.277235,31.376800,32.503188,32.881265,34.098839,35.203473,...,66.186159,66.400664,67.839584,71.356353,71.423614,74.145509,74.734795,80.793853,97.715924,145.631711
3,24.515624,36.132196,38.963373,39.124472,40.296619,41.367734,42.958551,43.173757,44.016935,45.308066,...,74.012528,73.701185,74.689867,76.990115,78.653697,83.350450,85.337845,91.069190,96.335752,135.294456
4,32.543605,43.099942,46.777572,47.405405,48.122471,48.375021,49.848134,50.328047,51.994145,53.288324,...,95.789329,96.008523,97.965432,99.923849,102.536319,108.256785,108.455675,118.229689,127.486370,171.736522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,96.600651,136.139638,142.490132,143.059855,146.335154,144.272832,149.565768,150.293764,157.006003,162.237078,...,275.246064,283.821334,280.642842,278.477819,289.458469,291.002691,302.194001,316.209714,364.321662,592.740369
354,81.281000,110.839949,118.265449,117.549283,119.608079,116.739332,122.987536,123.048643,126.763348,130.877779,...,178.059249,182.152100,176.564386,169.669085,181.188858,179.457694,187.173774,190.316539,203.265943,323.143043
355,78.700786,97.190693,108.880090,107.950350,108.011977,106.110956,110.012760,109.636602,110.843516,115.270711,...,112.099858,111.809541,108.416378,106.270372,117.975014,120.135963,129.418834,135.961543,144.681629,188.547165
356,80.858366,105.294996,112.575802,113.393661,115.069960,118.732790,119.936280,120.017358,124.629843,129.585368,...,167.950064,166.194044,170.572237,169.018702,177.080903,186.436446,191.888044,204.449042,264.832459,373.871582


# First setting: Q1+Q2

##  Gradient Boosting 1

In [9]:
CRPS(predictions_GB1.iloc[0:174], y_test[0:174])

13.207413793103454

In [7]:
pinball_sum = 0
for (index, colname) in enumerate(predictions_GB1.iloc[0:174]):
    pinball_sum += mean_pinball_loss(y_test[0:174], 
                  predictions_GB1.iloc[0:174][colname], 
                  alpha=(index+1)/100)
pinball_sum/99

1.5999453954904526

In [10]:
mean_pinball_loss(y_test[0:174], 
                  predictions_GB1.iloc[0:174]["0.05"], 
                  alpha=0.05)

1.5389535296936268

In [11]:
mean_pinball_loss(y_test[0:174], 
                  predictions_GB1.iloc[0:174]["0.50"], 
                  alpha=0.50)

2.61019064179013

In [12]:
mean_pinball_loss(y_test[0:174], 
                  predictions_GB1.iloc[0:174]["0.95"], 
                  alpha=0.95)

0.5945171318070203

## Gradient Boosting 2

In [13]:
CRPS(predictions_GB2.iloc[0:174], y_test[0:174])

15.940402298850563

In [8]:
pinball_sum = 0
for (index, colname) in enumerate(predictions_GB2.iloc[0:174]):
    pinball_sum += mean_pinball_loss(y_test[0:174], 
                  predictions_GB2.iloc[0:174][colname], 
                  alpha=(index+1)/100)
pinball_sum/99

1.7778540989029892

In [14]:
mean_pinball_loss(y_test[0:174], 
                  predictions_GB2.iloc[0:174]["0.05"], 
                  alpha=0.05)

1.5870472801325841

In [15]:
mean_pinball_loss(y_test[0:174], 
                  predictions_GB2.iloc[0:174]["0.50"], 
                  alpha=0.50)

2.5279021901473886

In [16]:
mean_pinball_loss(y_test[0:174], 
                  predictions_GB2.iloc[0:174]["0.95"], 
                  alpha=0.95)

1.2750414113492556

## LQR - Model 1

In [17]:
CRPS(predictions_model1.iloc[0:174], y_test[0:174])

16.21235632183909

In [9]:
pinball_sum = 0
for (index, colname) in enumerate(predictions_model1.iloc[0:174]):
    pinball_sum += mean_pinball_loss(y_test[0:174], 
                  predictions_model1.iloc[0:174][colname], 
                  alpha=(index+1)/100)
pinball_sum/99

3.8491819264259957

In [20]:
mean_pinball_loss(y_test[0:174], 
                  predictions_model1.iloc[0:174]["tau= 0.05"], 
                  alpha=0.05)

1.2641754079009961

In [21]:
mean_pinball_loss(y_test[0:174], 
                  predictions_model1.iloc[0:174]["tau= 0.50"], 
                  alpha=0.50)

5.263316949680461

In [22]:
mean_pinball_loss(y_test[0:174], 
                  predictions_model1.iloc[0:174]["tau= 0.95"], 
                  alpha=0.95)

2.2140312498370807

## LQR - Model 2

In [23]:
CRPS(predictions_model2.iloc[0:174], y_test[0:174])

18.300632183908043

In [10]:
pinball_sum = 0
for (index, colname) in enumerate(predictions_model2.iloc[0:174]):
    pinball_sum += mean_pinball_loss(y_test[0:174], 
                  predictions_model2.iloc[0:174][colname], 
                  alpha=(index+1)/100)
pinball_sum/99

4.6248063731358515

In [24]:
mean_pinball_loss(y_test[0:174], 
                  predictions_model2.iloc[0:174]["tau= 0.05"], 
                  alpha=0.05)

1.4765010862684647

In [25]:
mean_pinball_loss(y_test[0:174], 
                  predictions_model2.iloc[0:174]["tau= 0.50"], 
                  alpha=0.50)

6.576343937116396

In [26]:
mean_pinball_loss(y_test[0:174], 
                  predictions_model2.iloc[0:174]["tau= 0.95"], 
                  alpha=0.95)

2.3536576098009205

## LQR - Model 3

In [27]:
CRPS(predictions_model3.iloc[0:174], y_test[0:174])

19.373850574712648

In [11]:
pinball_sum = 0
for (index, colname) in enumerate(predictions_model3.iloc[0:174]):
    pinball_sum += mean_pinball_loss(y_test[0:174], 
                  predictions_model3.iloc[0:174][colname], 
                  alpha=(index+1)/100)
pinball_sum/99

2.951678138962135

In [28]:
mean_pinball_loss(y_test[0:174], 
                  predictions_model3.iloc[0:174]["tau= 0.05"], 
                  alpha=0.05)

1.0551209116845734

In [29]:
mean_pinball_loss(y_test[0:174], 
                  predictions_model3.iloc[0:174]["tau= 0.50"], 
                  alpha=0.50)

3.8663899347486703

In [30]:
mean_pinball_loss(y_test[0:174], 
                  predictions_model2.iloc[0:174]["tau= 0.95"], 
                  alpha=0.95)

2.3536576098009205

# Second Setting

##  Gradient Boosting 1

In [90]:
CRPS(predictions_GB1.iloc[175:], y_test[175:])

19.01073770491803

In [15]:
pinball_GB1_test2 = []
for (index, colname) in enumerate(predictions_GB1.iloc[175:]):
    pinball_GB1_test2.append(mean_pinball_loss(y_test[175:], 
                  predictions_GB1.iloc[175:][colname], 
                  alpha=(index+1)/100))

## Gradient Boosting 2

In [97]:
CRPS(predictions_GB2.iloc[175:], y_test[175:])

18.951174863387998

In [18]:
pinball_GB2_test2 = []
for (index, colname) in enumerate(predictions_GB2.iloc[175:]):
    pinball_GB2_test2.append(mean_pinball_loss(y_test[175:], 
                  predictions_GB2.iloc[175:][colname], 
                  alpha=(index+1)/100))

## LQR - Model 1

In [75]:
CRPS(predictions_model1.iloc[175:], y_test[175:])

21.350409836065577

In [28]:
pinball_model1_test2 = []
for (index, colname) in enumerate(predictions_model1.iloc[175:]):
    pinball_model1_test2.append(mean_pinball_loss(y_test[175:], 
                  predictions_model1.iloc[175:][colname], 
                  alpha=(index+1)/100))
pinball_model1_test2

[1.221210129799651,
 2.1943512269591623,
 3.0505614423552956,
 3.918789429095255,
 4.710157218340248,
 5.5493784657088066,
 6.230484458801871,
 7.029193842626726,
 7.541301077192582,
 8.033419052010839,
 8.656071562629233,
 9.148027993374907,
 9.59423259112029,
 10.010395276807381,
 10.602612987667683,
 11.114804027426327,
 11.661784046357939,
 12.06907485173903,
 12.6446679181401,
 13.212835376791535,
 13.668422162441203,
 14.121688933100211,
 14.660914567605548,
 14.921215519282837,
 15.379914843048038,
 15.904074682316098,
 16.187963951979906,
 16.39919159733168,
 16.74849843616291,
 17.328678854087045,
 17.649866943210736,
 18.05263117186218,
 18.33796456721357,
 18.69200491323435,
 19.096338540865226,
 19.495221745244045,
 20.032422359714054,
 20.56987293121862,
 20.89354608072023,
 21.23712232854064,
 21.336310834465113,
 21.437272049765543,
 21.38832119078853,
 21.48777432478533,
 21.80204168842814,
 21.783738559656694,
 21.904836810484433,
 21.84509604597264,
 21.84677000402887

## LQR - Model 2

In [81]:
CRPS(predictions_model2.iloc[175:], y_test[175:])

23.519808743169378

In [21]:
pinball_model2_test2 = []
for (index, colname) in enumerate(predictions_model2.iloc[175:]):
    pinball_model2_test2.append(mean_pinball_loss(y_test[175:], 
                  predictions_model2.iloc[175:][colname], 
                  alpha=(index+1)/100))

## LQR - Model 3

In [85]:
CRPS(predictions_model3.iloc[175:], y_test[175:])

23.89827868852457

In [22]:
pinball_model3_test2 = []
for (index, colname) in enumerate(predictions_model3.iloc[175:]):
    pinball_model3_test2.append(mean_pinball_loss(y_test[175:], 
                  predictions_model3.iloc[175:][colname], 
                  alpha=(index+1)/100))

In [26]:
pinball_model1_test2

[]

In [35]:
pinball2 = pd.DataFrame({
    "tau": np.linspace(1,99, num=99),
    "model1": pinball_model1_test2,
    "model2": pinball_model2_test2,
    "model3": pinball_model3_test2,
    "GB1": pinball_GB1_test2,
    "GB2": pinball_GB2_test2,
})

In [37]:
pinball2.to_csv("../../results/pinball2.csv")